In [1]:
# System imports
from datetime import datetime

# Third-party imports
import yfinance as yf
from pathlib import Path

# Local imports
from portfolio.holdings import Portfolio
import datehandler
from portfolio.commsec import Trades

In [0]:
portfolio = Portfolio()
df = portfolio.daily_holdings

In [0]:
df

In [0]:
stock = yf.Ticker('ZNO.AX')
stock.history(period='1y')

In [0]:
portfolio._calculate_value(df)